# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [2]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
openai = OpenAI()

API key looks good so far


In [4]:
# set up environment

question_system_prompt = "You are provided with a technical question about AI and LLMs \
Provide the most accurate answer and explain in detail to a student of LLM engineering the answer \n"


In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [8]:
# Get gpt-4o-mini to answer, with streaming

def stream_question_answer(question):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": question_system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

stream_question_answer(question)

Let's break down the provided code snippet step by step, especially in the context of Python, which appears to be the language in use.

python
yield from {book.get("author") for book in books if book.get("author")}


### Detailed Explanation:

1. **Context of `yield`**:
   - The `yield` keyword is part of Python's generator functionality. When you use `yield`, the function becomes a generator, allowing it to produce a series of values over time instead of computing them all at once and returning them.
   - In this context, `yield from` is used to delegate part of the generator's operations to another generator or iterable. It effectively yields all values from the iterable without needing to loop through it explicitly.

2. **Set comprehension**:
   - `{book.get("author") for book in books if book.get("author")}` is a set comprehension. This is a concise and readable way to create a set from the `books` collection.
   - The expression iterates through a collection named `books`, which we can assume is a list (or some other iterable) of dictionaries (each dictionary representing a book).

3. **What does `book.get("author")` do?**:
   - In each iteration, `book.get("author")` retrieves the value associated with the key `"author"` from each dictionary (representing a book). If the author key does not exist in the dictionary, `get` will return `None` instead of raising a KeyError.
   - The `if book.get("author")` condition filters out any books that do not have an author defined. If the `get` method returns `None`, that book will not be included in the new set. This means only books with valid author names will be processed.

4. **Result - Set of Authors**:
   - The result of the set comprehension is a set of unique author names from the `books` collection. Since it’s a set, any duplicate author names will be automatically removed.

5. **Final Output of `yield from`**:
   - Using `yield from` here allows the generator to yield each unique author name from the set one by one. It effectively pulls values from the set and yields them to whatever is consuming this generator. 

### Summary Example:

Suppose the `books` collection looks like this:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3"},  # No author
    {"title": "Book 4", "author": "Author A"},  # Duplicate author
    {"title": "Book 5", "author": "Author C"}
]


When you run the line of code, the resulting set comprehension `{book.get("author") for book in books if book.get("author")}` would produce:

python
{"Author A", "Author B", "Author C"}


When the generator yields from this set, it will yield `Author A`, then `Author B`, and finally `Author C`.

### Conclusion:
In summary, the code snippet creates a generator that yields each unique author from a list of books, ensuring that only those books that have specified authors are included, filtering out any that don't. The use of `yield from` makes this process efficient and keeps the code clean and readable.

In [11]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [12]:
# Get Llama 3.2 to answer

def stream_question_answer_local(question):
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": question_system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

stream_question_answer_local(question)

**Breaking Down the Code**

This line of code is written in Python and utilizes a feature called "async generator" or "coroutine" with yield from. Here's a breakdown of what it does:

- `yield from`: This keyword allows us to delegate part of the execution of a coroutine to another asynchronous generator.
- `{book.get("author") for book in books if book.get("author")}`: This is an asynchronous generator expression, which behaves similarly to a list comprehension. However, instead of evaluating an expression and returning it immediately, an asynchronous generator yields control to itself between each iteration.

Let's break down the logic:

1. `for book in books`: Iterates over some collection (e.g., a list or dictionary) called `books`. Each item in this collection is expected to be another iterable like a dictionary or something that doesn't matter for the comprehension.

2. `if book.get("author")`: Before considering each item, checks whether it has a key named `"author"`. If not, this item will be skipped.

3. `yield from ...`: The items in this comprehension (which is basically filtering and iterating over `book`) when we use yield from the generator becomes to get one element on a loop calling  it inside where to call the code inside if the value passed into

In terms of what it yields, consider what it does without yield from:

`{item: book dictionary for book in books if book.get("author")} => [{'author': author} for book in books if book.get('author')]` 

Now if we use `yield from {book.get("author") for book in books if book.get("author")}`

   - Yield items until no item is left.

So what's that?

yield from allows to turn those comprehension into an yield one. By using the key `.get()` which gets the key if its exists else will returns `None` this can lead us to a  code like:
  

 
    async function books():
      const authors = await (yield from {book.get("author") for book in books if book.get("author")});

      return authors;

Here you would get all values for the `'author'` key and the function gets to run and wait for them before returning.

javascript
const books = async () => {
  const authors = await ( yield from {book.get("author") for book in books if book.get("author")} );

  return authors;
}

books();
